In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm as anova

def EDA_plots(df, features = df.columns):
    for feature in features:
        if feature != 'SalePrice' and feature != 'PID':
            print('feature: ',feature)
            scatter = px.scatter(x = df[f'{feature}'], y = df['SalePrice'])
            scatter.update_layout(
                title={
                    'text': f'Scatterplot, {feature} vs SalePrice',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'SalePrice'
            )
            scatter.show()
            hist = px.histogram(x = df[f'{feature}'])
            hist.update_layout(
                title={
                    'text': f'Distribution of {feature}',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'Frequency'
            )
            hist.show()
            box = px.box(x = df[f'{feature}'], y = df['SalePrice'])
            box.update_layout(
                title={
                    'text': f'Boxplot, {feature} vs SalePrice',
                    'y':0.95,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                xaxis_title = f'{feature}',
                yaxis_title = 'Frequency'
            )
            box.show()
            temp = df[df[f'{feature}'].isna() == False].reset_index(drop = True)
            if type(temp.loc[0, f'{feature}']) != str:
                price_corr = temp[f'{feature}'].corr(temp['SalePrice'])
                print(f'Correlation between {feature} and sale price is {price_corr}')
                linreg = stats.linregress(temp[f'{feature}'], temp['SalePrice'] )
                print(linreg)
                print('r^2 = ',linreg.rvalue**2)
            if type(temp.loc[0, f'{feature}']) == str:
                fit = ols(f'SalePrice ~ C({feature})', data=temp).fit()
                anova_table = anova(fit, typ=2)
                print(anova_table)
        print()